In [11]:
import pandas as pd
import re

file = pd.read_csv('ggLink.csv', sep='\t')
print(file[['gene1','gene2','minResCount']])

         gene1  gene2  minResCount
0            2     C2            3
1            2     C3            8
2            2    C4B            8
3            2  C4BPA            2
4            2     C5            8
...        ...    ...          ...
1322480  ZWINT  ZWINT         3828
1322481   ZXDA   ZXDA            5
1322482   ZXDA   ZXDC            5
1322483   ZXDC   ZXDC            5
1322484    ZYX    ZYX            6

[1322485 rows x 3 columns]


In [16]:
def remove_version(x):
    if '.' in x:
        return x.split('.')[0]
    return x

gtf_file_path = '/work/h2020deciderficarra_shared/TCGA/OV/project_n16_data/gencode.v47.annotation.gtf'
gtf = pd.read_csv(gtf_file_path, sep="\t", header=None, comment='#')
gtf.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']

parameters = ['gene_id', 'gene_type', 'gene_name']
for p in parameters:
    gtf[p] = gtf['attribute'].apply(lambda x: re.findall(rf'{p} "([^"]*)"', x)[0] if rf'{p} "' in x else np.nan)

gtf.drop('attribute', axis=1, inplace=True)

gtf['gene_id'] = gtf['gene_id'].apply(remove_version)

gtf_pc = gtf[gtf['gene_type'] == 'protein_coding']

# Protein coding set
pc_set = set(gtf_pc['gene_id'].to_list())


In [30]:

gtf_pc[gtf_pc['gene_name'] == 'OR4F5'].iloc[0]

seqname                 chr1
source                HAVANA
feature                 gene
start                  65419
end                    71585
score                      .
strand                     +
frame                      .
gene_id      ENSG00000186092
gene_type     protein_coding
gene_name              OR4F5
Name: 2486, dtype: object